In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.message import cm

In [ ]:
from component.model.recipe import Recipe
from component.tile.custom_aoi_tile import AoiTile
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

In [ ]:
build_alert = AlertState()

In [ ]:
build_alert

In [ ]:
recipe = Recipe()
recipe.load_model()

In [ ]:
aoi_tile = AoiTile()

In [ ]:
dashboard_tile = DashboardTile()
dashboard_tile.build(recipe=recipe, build_alert=build_alert)

In [ ]:
aoi_tile.build(recipe=recipe, build_alert=build_alert)

In [ ]:
aoi_tile

In [ ]:
dashboard_tile

## Run the dashboard

In [ ]:
wlc_outputs = "wlc_out", "benefit_list", "constraint_list", "cost_list"

In [ ]:
ee_features = aoi_model.get_ee_features()

# retreive the area and theme json result
cs.get_stats(
    ee_features,
    wlc_outputs,
    layer_model,
)

In [ ]:
stats = [
    get_summary_statistics(wlc_outputs, name, geom, layer_model.layer_list, True)
    for name, geom in enumerate(ee_features)
]

## Calculate WLC

- get constraints mask
- get the benefit index
- get the benefit/cost index
- get constraint_index (this is WLC)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.model import BenefitModel
from component.model import ConstraintModel
from component.model import CostModel
from component.model import SeplanAoi

from component.scripts.seplan import Seplan, reduce_constraints
from component.scripts.statistics import get_summary_statistics
from component.tile.map_tile import MapTile
from component.tile.questionnaire_tile import QuestionnaireTile

from component import tile

In [ ]:
seplan_aoi = SeplanAoi()

seplan_aoi.aoi_model.method = "ADMIN1"
seplan_aoi.aoi_model.admin = "959"
seplan_aoi.aoi_model.set_object()

benefit_model = BenefitModel()
constraint_model = ConstraintModel()
cost_model = CostModel()


class Recipe:

    seplan = Seplan(
        seplan_aoi,
        benefit_model,
        constraint_model,
        cost_model,
    )


recipe = Recipe()

### Test stats 

In [ ]:
ee_features = recipe.seplan.aoi_model.get_ee_features()
ee_features

In [ ]:
from component.tile.dashboard_tile import DashboardTile
from component.widget.alert_state import AlertState

In [ ]:
alert = AlertState()
alert

In [ ]:
dashboard_tile = DashboardTile()

In [ ]:
dashboard_tile.build(recipe, alert)

In [ ]:
dashboard_tile

In [ ]:
summary_stats = get_summary_statistics(recipe.seplan)

In [ ]:
from component.scripts.statistics import parse_theme_stats

In [ ]:
summary_stats

In [ ]:
parse_theme_stats(summary_stats)

## Calculate the image percent cover pixel area

In [ ]:
from component.scripts.statistics import get_image_percent_cover_pixelarea
from component.scripts.seplan import mask_image

In [ ]:
aoi = seplan_model.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.constraint_model.assets[0]
data_type = recipe.seplan.constraint_model.data_type[0]
maskout_values = recipe.seplan.constraint_model.values[0]

asset_id, data_type, maskout_values

In [ ]:
# maskout any image from the model

masked_image = mask_image(asset_id, data_type, maskout_values)

In [ ]:
get_image_percent_cover_pixelarea(masked_image, aoi, "perro").getInfo()

In [ ]:
from sepal_ui.mapping import SepalMap
import ee

ee.Initialize()

In [ ]:
map_ = SepalMap()
map_.addLayer(masked_image.unmask(0).randomVisualizer().clip(aoi))
map_

## Calculate the image sum on COSTS

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from component.scripts.statistics import get_image_sum
from component.scripts.seplan import reduce_constraints, asset_to_image

In [ ]:
aoi = recipe.seplan.aoi_model.feature_collection

In [ ]:
asset_id = recipe.seplan.cost_model.assets[0]
constraint_list = recipe.seplan.get_masked_constraints_list()
mask_out_areas = reduce_constraints(constraint_list)

In [ ]:
image = asset_to_image(asset_id)

In [ ]:
sum_img = image.updateMask(mask_out_areas).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=aoi,
    scale=100,
    maxPixels=1e13,
)
sum_img.values().get(0).getInfo()

In [ ]:
get_image_sum(image, aoi, mask_out_areas, "name").getInfo()

In [ ]:
area_ha = (
    ee.Image.pixelArea()
    .divide(10000)
    .reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=100,
        maxPixels=1e13,
    )
    .getInfo()
)

In [ ]:
1227466904.7257729 / 403841.9645242746

In [ ]:
area_ha

In [ ]:
1227466904.7257729 / (2379515721.649737 + 1227466904.7257729)

In [ ]:
map_.addLayer(constraint_list[0][0].clip(aoi), {}, "mask")

In [ ]:
map_ = SepalMap()
map_.addLayer(mask_out_areas.randomVisualizer().clip(aoi))
map_